In [36]:
#Load the dataset and explore the variables.

In [49]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import warnings
import imblearn
import pymysql
from sqlalchemy import create_engine
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.metrics import confusion_matrix
warnings.filterwarnings('ignore')

In [50]:
data = pd.read_csv(r"C:\Users\a_del\Downloads\Customer-Churn.csv")
data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [51]:
data.isna().sum()
    
    


gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [52]:
data.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [53]:
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce') #changing from categorical to numeric 

In [54]:
data.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7032.000000
mean,0.162147,32.371149,64.761692,2283.300441
std,0.368612,24.559481,30.090047,2266.771362
min,0.000000,0.000000,18.250000,18.800000
25%,0.000000,9.000000,35.500000,401.450000
50%,0.000000,29.000000,70.350000,1397.475000
75%,0.000000,55.000000,89.850000,3794.737500
max,1.000000,72.000000,118.750000,8684.800000


In [55]:
#We will try to predict variable Churn using a logistic regression on variables tenure, SeniorCitizen,MonthlyCharges.

In [56]:
data1 = data[['tenure', 'SeniorCitizen', 'MonthlyCharges', 'Churn']]#new dataframe with variales and target

In [57]:
##Extract the target variable and the independent variables and scale them.

In [58]:
X = pd.get_dummies(data1.drop('Churn', axis=1))
y = data1['Churn']

In [59]:
scale = StandardScaler()

scaled_X = scale.fit_transform(X)

In [60]:
#Build the logistic regression model.

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

model = LogisticRegression(random_state=40, max_iter=10000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=10000, random_state=40)

In [ ]:
#Evaluate the model

In [62]:
predictions = model.predict(X_test)
model.score(X_test, y_test)

0.7785663591199432

In [63]:
#Even a simple model will give us more than 70% accuracy. Why?

In [ ]:
#Because the date is imblanaced?

In [ ]:
#As the data is imbalnced we will try first SMOTE and then TOMEK LINKS

In [ ]:
#Synthetic Minority Oversampling TEchnique (SMOTE) is an over sampling technique based on nearest neighbors that adds new points between existing points. 
#Apply imblearn.over_sampling.SMOTE to the dataset. Build and evaluate the logistic regression model. Is it there any improvement?

In [ ]:
#SMOTE

In [64]:
smote = SMOTE()

X_train, y_train = smote.fit_resample(X_train, y_train)

y_train.value_counts()

No     4140
Yes    4140
Name: Churn, dtype: int64

In [65]:
model = LogisticRegression(random_state = 40, max_iter = 1000)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
model.score(X_test, y_test)

0.7040454222853088

In [ ]:
#Tomek links are pairs of very close instances, but of opposite classes. 
#Removing the instances of the majority class of each pair increases the space between the two classes, facilitating the classification process. 
#Apply imblearn.under_sampling.TomekLinks to the dataset. 
#Build and evaluate the logistic regression model. Is it there any improvement?

In [66]:
#Tomeklinks

In [68]:
tomek = TomekLinks()

X = pd.get_dummies(data1.drop('Churn', axis=1))
y = data1['Churn']

X_tl, y_tl = tomek.fit_resample(X, y)

y_tl.value_counts()

No     4712
Yes    1869
Name: Churn, dtype: int64

In [69]:
model = LogisticRegression(random_state = 40, max_iter = 1000)
model.fit(X_tl, y_tl)

predictions = model.predict(X_tl)
model.score(X_tl, y_tl)

0.7910651876614496

In [ ]:
#the score improved only with Tomeklinks